In [1]:
import numpy as np
import doctest
import random

In [2]:
def directionToPosition( start, direction ):
    """
    >>> directionToPosition( [1, 1], 1 )
    array([1, 2])
    >>> directionToPosition( [0, 0], 0 )
    array([0, 0])
    >>> directionToPosition( [0, 3], 1 )
    array([0, 3])
    >>> directionToPosition( [3, 0], 2 )
    array([3, 0])
    >>> directionToPosition( [0, 0], 3 )
    array([0, 0])
    """
    if ( start[ 0 ] == 0 and start[ 1 ] == 0 ) or ( start[ 0 ] == 3 and start[ 1 ] == 3 ):
        return np.copy( start )
    directions = [
        [ -1,  0 ],
        [  0, +1 ],
        [ +1,  0 ],
        [  0, -1 ]
    ]
    start = np.copy( start )
    end = np.copy( start )
    end[ 0 ] += directions[ direction ][ 0 ]
    end[ 1 ] += directions[ direction ][ 1 ] 
    if end[ 0 ] < 0 or end[ 0 ] > 3 or end[ 1 ] < 0 or end[ 1 ] > 3:
        return start
    return end
doctest.testmod()

TestResults(failed=0, attempted=5)

## Iteratative Policy Evaluation

In [3]:
leave_costs = np.full( ( 4, 4 ), -1 )
leave_costs[ 0, 0 ] = 0
leave_costs[ 3, 3 ] = 0

def randomPolicy( position ):
    return np.full( ( 4 ), 0.25 )

def evaluatePolicy( policy, k_max=4, print_intermediate=False, v_k=None, in_place=False ):
    """
    Evaluates a policy
    :param policy: Function that returns the chance of taking each action
    """
    if v_k is None:
        v_k = np.zeros( (4, 4), dtype=np.float )
    for k in range( k_max ):
        if print_intermediate:
            print( v_k )
        v_new = v_k
        if not in_place:
            v_new = v_k.copy()
        # Go through all states
        for x in range( 4 ):
            for y in range( 4 ):
                choice = policy( np.array( [ x, y ] ) )
                total_reward = 0.0
                for direction in range( 4 ):
                    new_position = directionToPosition( np.array( [ x, y ] ), direction )
                    total_reward += choice[ direction ] * ( leave_costs[ x, y ] + v_k[ new_position[ 0 ], new_position[ 1 ] ] )
                v_new[ x, y ] = total_reward
        v_k = v_new
    return v_k

v_k_4 = evaluatePolicy( randomPolicy, 4, True )

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
[[ 0.   -1.75 -2.   -2.  ]
 [-1.75 -2.   -2.   -2.  ]
 [-2.   -2.   -2.   -1.75]
 [-2.   -2.   -1.75  0.  ]]
[[ 0.     -2.4375 -2.9375 -3.    ]
 [-2.4375 -2.875  -3.     -2.9375]
 [-2.9375 -3.     -2.875  -2.4375]
 [-3.     -2.9375 -2.4375  0.    ]]


## Policy Iteration

In [4]:
def generateGreedyPolicy( v_k ):
    """
    >>> v_k_test = evaluatePolicy( randomPolicy, 4 )
    >>> greedy_policy = generateGreedyPolicy( v_k_test )
    >>> greedy_policy( np.array( [ 1, 1 ] ) )
    array([0.5, 0. , 0. , 0.5])
    """
    def greedyPolicy( position ):
        direction_values = np.zeros( ( 4 ), dtype=np.float )
        for direction in range( 4 ):
            new_position = directionToPosition( position, direction )
            direction_values[ direction ] = v_k[ new_position[ 0 ], new_position[ 1 ] ]
        choices = np.zeros( ( 4 ), dtype=np.float )

        choices[ direction_values == direction_values.max() ] = 1
        total_choices = np.count_nonzero( choices )
        choices[ choices == 1 ] /= total_choices

        return choices
    return greedyPolicy
doctest.testmod()

k_max = 10
V = evaluatePolicy( randomPolicy, k_max=k_max )
for i in range( 3 ):
    pi = generateGreedyPolicy( V )
    V = evaluatePolicy( randomPolicy, k_max=k_max )
print( V )

[[ 0.         -6.13796997 -8.35235596 -8.96731567]
 [-6.13796997 -7.73739624 -8.42782593 -8.35235596]
 [-8.35235596 -8.42782593 -7.73739624 -6.13796997]
 [-8.96731567 -8.35235596 -6.13796997  0.        ]]


## Value Iteration

> (k_max=1)

In [5]:
leave_costs[ 3, 3 ] = -1
# Make gridworld the Value Iteration example from the lecture

In [6]:
k_max = 1
V = evaluatePolicy( randomPolicy, k_max=k_max )
for i in range( 5 ):
    print( V )
    pi = generateGreedyPolicy( V )
    V = evaluatePolicy( pi, k_max=k_max, v_k=V )
print( V )

[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]]
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -2.]
 [-2. -2. -2. -2.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -3.]
 [-2. -3. -3. -3.]
 [-3. -3. -3. -3.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -4.]
 [-3. -4. -4. -4.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -5.]
 [-3. -4. -5. -5.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -5.]
 [-3. -4. -5. -6.]]


## Extensions: In-Place Value Iteration 

In [7]:
k_max = 1
V = evaluatePolicy( randomPolicy, k_max=k_max )
for i in range( 5 ):
    print( V )
    pi = generateGreedyPolicy( V )
    V = evaluatePolicy( pi, k_max=k_max, v_k=V, in_place=True )
print( V )

[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]]
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -2.]
 [-2. -2. -2. -2.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -3.]
 [-2. -3. -3. -3.]
 [-3. -3. -3. -3.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -4.]
 [-3. -4. -4. -4.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -5.]
 [-3. -4. -5. -5.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -5.]
 [-3. -4. -5. -6.]]
